In [1]:
# Database Connection
import sqlalchemy
from sqlalchemy import create_engine, inspect, text

# Data Science
import pandas as pd
import numpy as np

# Data Vizualization
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Assign SQL variables
SQL_USERNAME = "postgres"
SQL_PASSWORD = "postgres" # change this
SQL_IP = "localhost"
PORT = 5432
DATABASE = "project_two_etl" # change this

In [3]:
# Set up Engine
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

In [4]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Use the inspector to print the column names within each table and its data types
for table in tables:
    print(table)
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])
        
    print()

contacts
contact_id INTEGER
first_name VARCHAR(100)
last_name VARCHAR(250)
email VARCHAR(250)
last_updated TIMESTAMP

campaign
cf_id INTEGER
contact_id INTEGER
company_name VARCHAR(100)
description VARCHAR(500)
goal INTEGER
pledged INTEGER
outcome VARCHAR(100)
backers_count INTEGER
country VARCHAR(5)
currency VARCHAR(5)
launched_date DATE
end_date DATE
category_id VARCHAR(5)
subcategory_id VARCHAR(100)
last_updated TIMESTAMP

category
category_id VARCHAR(5)
category VARCHAR(100)
last_updated TIMESTAMP

subcategory
subcategory_id VARCHAR(8)
subcategory VARCHAR(100)
last_updated TIMESTAMP



In [8]:
# Retrieve full contacts table from Postgres
query = """
    SELECT *
    FROM contacts
    """
contacts = pd.read_sql(text(query), con=engine)
contacts.head()

,contact_id,first_name,last_name,email,last_updated
0,4661,Cecilia,Velasco,cecilia.velasco@rodrigues.fr,2024-07-16 15:26:49.529022
1,3765,Mariana,Ellis,mariana.ellis@rossi.org,2024-07-16 15:26:49.529022
2,4187,Sofie,Woods,sofie.woods@riviere.com,2024-07-16 15:26:49.529022
3,4941,Jeanette,Iannotti,jeanette.iannotti@yahoo.com,2024-07-16 15:26:49.529022
4,2199,Samuel,Sorgatz,samuel.sorgatz@gmail.com,2024-07-16 15:26:49.529022


In [9]:
# Retrieve full category table from Postgres
query = """
    SELECT *
    FROM category
    """
category = pd.read_sql(text(query), con=engine)
category.head()

,category_id,category,last_updated
0,cat1,food,2024-07-16 15:27:52.504756
1,cat2,music,2024-07-16 15:27:52.504756
2,cat3,technology,2024-07-16 15:27:52.504756
3,cat4,theater,2024-07-16 15:27:52.504756
4,cat5,film & video,2024-07-16 15:27:52.504756


In [10]:
# Retrieve full subcategory table from Postgres
query = """
    SELECT *
    FROM subcategory
    """
subcategory = pd.read_sql(text(query), con=engine)
subcategory.head()

,subcategory_id,subcategory,last_updated
0,subcat1,food trucks,2024-07-16 15:28:18.332769
1,subcat2,rock,2024-07-16 15:28:18.332769
2,subcat3,web,2024-07-16 15:28:18.332769
3,subcat4,plays,2024-07-16 15:28:18.332769
4,subcat5,documentary,2024-07-16 15:28:18.332769


In [11]:
# Retrieve full campaign table from Postgres
query = """
    SELECT *
    FROM campaign
    """
campaign = pd.read_sql(text(query), con=engine)
campaign.head()

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,category_id,subcategory_id,last_updated
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1970-01-01,1970-01-01,cat1,subcat1,2024-07-16 15:29:04.161966
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1970-01-01,1970-01-01,cat2,subcat2,2024-07-16 15:29:04.161966
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1970-01-01,1970-01-01,cat3,subcat3,2024-07-16 15:29:04.161966
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1970-01-01,1970-01-01,cat2,subcat2,2024-07-16 15:29:04.161966
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1970-01-01,1970-01-01,cat4,subcat4,2024-07-16 15:29:04.161966


In [ ]:
# Close Connection
engine.dispose()